In [ ]:
import sys
import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
import cell2location
import scvi
from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42 # enables correct plotting of text
import seaborn as sns

In [ ]:
results_folder = './results/A1neural/'
ref_run_name = f'{results_folder}/reference_signatures'
run_name = f'{results_folder}/cell2location_map'

In [ ]:
adata_vis = sc.read_visium("./rawfiles/A1/")
adata_vis.var_names_make_unique()
adata_vis.var["mt"] = adata_vis.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata_vis, qc_vars=["mt"], inplace=True)
sc.pp.filter_cells(adata_vis, min_counts=500)
print(f"#cells after min filter: {adata_vis.n_obs}")
sc.pp.filter_cells(adata_vis, max_counts=18000)
print(f"#cells after max filter: {adata_vis.n_obs}")
adata_vis = adata_vis[adata_vis.obs["pct_counts_mt"] < 20]
print(f"#cells after MT filter: {adata_vis.n_obs}")
sc.pp.filter_genes(adata_vis, min_cells=10)
adata_vis.obs['sample'] = list(adata_vis.uns['spatial'].keys())[0]

In [ ]:
adata_ref = sc.read_h5ad("neural.h5ad")


In [ ]:
scvi.data.setup_anndata(adata=adata_ref,
                        batch_key='Sample',
                        labels_key='Cluster'
                       )
scvi.data.view_anndata_setup(adata_ref)

In [ ]:
from cell2location.models import RegressionModel
mod = RegressionModel(adata_ref)
mod.train(max_epochs=250, batch_size=2500, train_size=1, lr=0.001, use_gpu=False)


In [ ]:
adata_ref = mod.export_posterior(
    adata_ref, sample_kwargs={'num_samples': 1000, 'batch_size': 2500, 'use_gpu': False}
)

In [ ]:
if 'means_per_cluster_mu_fg' in adata_ref.varm.keys():
    inf_aver = adata_ref.varm['means_per_cluster_mu_fg'][[f'means_per_cluster_mu_fg_{i}'
                                    for i in adata_ref.uns['mod']['factor_names']]].copy()
else:
    inf_aver = adata_ref.var[[f'means_per_cluster_mu_fg_{i}'
                                    for i in adata_ref.uns['mod']['factor_names']]].copy()
inf_aver.columns = adata_ref.uns['mod']['factor_names']
inf_aver.iloc[0:5, 0:5]

In [ ]:
adata_vis.var['MT_gene'] = [gene.startswith('MT-') for gene in adata_vis.var.index]
adata_vis.obsm['MT'] = adata_vis[:, adata_vis.var['MT_gene'].values].X.toarray()
adata_vis = adata_vis[:, ~adata_vis.var['MT_gene'].values]

In [ ]:
intersect = np.intersect1d(adata_vis.var_names, inf_aver.index)
adata_vis = adata_vis[:, intersect].copy()
inf_aver = inf_aver.loc[intersect, :].copy()

In [ ]:
scvi.data.setup_anndata(adata=adata_vis, batch_key="sample")
scvi.data.view_anndata_setup(adata_vis)
mod = cell2location.models.Cell2location(
    adata_vis, cell_state_df=inf_aver,
    detection_alpha=200
)

mod.train(max_epochs=30000,
          batch_size=None,
          train_size=1,
          use_gpu=True)
mod.plot_history(1000)
plt.legend(labels=['full data training']);

In [ ]:
adata_vis = mod.export_posterior(
    adata_vis, sample_kwargs={'num_samples': 1000, 'batch_size': mod.adata.n_obs, 'use_gpu': True}
)

In [ ]:
adata_vis.obs[adata_vis.uns['mod']['factor_names']] = adata_vis.obsm['q05_cell_abundance_w_sf']

In [ ]:
mod.samples = adata_vis.uns['mod']
expected_dict = mod.module.model.compute_expected_per_cell_type(
    mod.samples["post_sample_q05"], mod.adata
)
for i, n in enumerate(mod.factor_names_):
    adata_vis.layers[n] = expected_dict['mu'][i]

In [ ]:
adata_vis.write('Adultneural.h5ad')